In [ ]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
from collections import Counter

In [ ]:
from pbpstats.client import Client

settings = {
    "Games": {"source": "web", "data_provider": "data_nba"},
     "dir": "G:/My Drive/Sra_coding/NBA/pbpdata",
}
client = Client(settings)
# ID of all games for 2020-21 Season
year = 2020
season = client.Season("nba", str(year)+'-'+str(year+1)[-2:], "Regular Season")
# season = client.Season("wnba", str(year), "Regular Season")
games_id = []
k = 0
for final_game in season.games.final_games:
    k += 1
    clear_output(wait=True)
    print(k)
    games_id.append(final_game['game_id'])

In [ ]:
settings = {
    "Boxscore": {"source": "file", "data_provider": "data_nba"},
    "Possessions": {"source": "file", "data_provider": "data_nba"},
    "dir": "G:/My Drive/Sra_coding/NBA/pbpdata"
}
client = Client(settings)
games_list = []
bad_games_list = []
k = 0
for gameid in games_id:
    clear_output(wait=True)
    k += 1
    print(k)
    try:
        games_list.append(client.Game(gameid))
    except:
        print(gameid)
        bad_games_list.append(gameid)
        continue

In [ ]:
len(bad_games_list)

In [113]:
events = games_list[0].possessions.items[0].events
dir(events[1])
events[2].player1_id
events

[<DataStartOfPeriod GameId: 0022000001, Description: Start Period, Time: 12:00, EventNum: 2>,
 <DataJumpBall GameId: 0022000001, Description: Jump Ball Jordan vs Wiseman, Time: 11:58, EventNum: 4>,
 <DataTurnover GameId: 0022000001, Description: [BKN] Jordan Turnover : Out of Bounds - Bad Pass Turnover (1 TO), Time: 11:50, EventNum: 7>]

In [96]:
from pbpstats.resources.enhanced_pbp import Rebound
from pbpstats.resources.enhanced_pbp import Turnover
from pbpstats.resources.enhanced_pbp import FieldGoal
from pbpstats.resources.enhanced_pbp import FreeThrow
pos = []
posdes = []
points = []
event_type = []
off1, off2, off3, off4, off5 = [],[],[],[],[]
def1, def2, def3, def4, def5 = [],[],[],[],[]
for game in games_list:
    hmscr = []
    rdscr = []
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if possession_event.count_as_possession and possession_event.event_type in (1,3,4,5):
                if possession_event.description == 'Game End':
                    continue
                off_id =  possession_event.get_offense_team_id()
                if not bool(off_id):
                    continue
                # off_id =  possession_event.team_id
                teams = list(possession_event.current_players.keys())
                for te in teams:
                    if te == off_id:
                        off1.append(possession_event.current_players[te][0])
                        off2.append(possession_event.current_players[te][1])
                        off3.append(possession_event.current_players[te][2])
                        off4.append(possession_event.current_players[te][3])
                        off5.append(possession_event.current_players[te][4])
                    else:
                        def1.append(possession_event.current_players[te][0])
                        def2.append(possession_event.current_players[te][1])
                        def3.append(possession_event.current_players[te][2])
                        def4.append(possession_event.current_players[te][3])
                        def5.append(possession_event.current_players[te][4])
                # pos.append(possession_event)
                posdes.append(possession_event.description)
                event_type.append(possession_event.event_type)
                hmscr.append(possession_event.home_score)
                rdscr.append(possession_event.away_score)
    hm_pts = []
    rd_pts = []
    for i in range(len(hmscr)-1):
        hm_pts.append(hmscr[i+1]-hmscr[i])
        rd_pts.append(rdscr[i+1]-rdscr[i])
    pts = []
    pts.append(max(hmscr[0],hmscr[0]))
    for i in range(len(hmscr)-1):
        pts.append(max(hm_pts[i],rd_pts[i]))
    points.extend(pts)

In [97]:
print(len(points))
print(len(off1))
print(len(def1))

209695
209695
209695


In [102]:
# data1 = pd.DataFrame({'Possession':posdec,'Home':hmscr,'Road':rdscr,'Points':pts})
data = pd.DataFrame({'off1':off1, 'off2':off2, 'off3':off3, 'off4':off4, 'off5':off5, 
                    'def1':def1, 'def2':def2, 'def3':def3, 'def4':def4, 'def5':def5,
                     'Points':points,'EventType':event_type}) #, 'Description':posdes
data.to_csv ('fdata/NBA_possessions_{0}.csv'.format(year),index=False)

In [101]:
409/209695

0.0019504518467297742

In [100]:
(1,3,4,5)

(1, 3, 4, 5)